In [1]:
from jac_misc.pdf_ext import *

In [2]:
pdf_ext.extract_pdf(url="http://www.africau.edu/images/default/sample.pdf", metadata=False)

{'pages': 2,
 'content': [' A Simple PDF File \n This is a small demonstration .pdf file - \n just for use in the Virtual Mechanics tutorials. More text. And more \n text. And more text. And more text. And more text. \n And more text. And more text. And more text. And more text. And more \n text. And more text. Boring, zzzzz. And more text. And more text. And \n more text. And more text. And more text. And more text. And more text. \n And more text. And more text. \n And more text. And more text. And more text. And more text. And more \n text. And more text. And more text. Even more. Continued on page 2 ...\n',
  ' Simple PDF File 2 \n ...continued from page 1. Yet more text. And more text. And more text. \n And more text. And more text. And more text. And more text. And more \n text. Oh, how boring typing this stuff. But not as boring as watching \n paint dry. And more text. And more text. And more text. And more text. \n Boring.  More, a little more text. The end, and just as well. \

In [6]:
from PyPDF2 import PdfFileReader

In [7]:
import requests
import os
import re
import uuid

from PyPDF2 import PdfFileReader

def download_pdf(url: str, filename: str):
    r"""method for downloading PDF from URL
    :param url: URL for the downloading a pdf file.
    :param filename: filename for saving downloaded pdf locally.
    """
    with requests.get(url, stream=True) as req:
        req.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in req.iter_content(chunk_size=2048):
                f.write(chunk)


def valid_url(url: str):
    r"""validating whether given url is a valid pdf url or not
    :param url: URL for the downloading a pdf file.
    """
    r = requests.get(url)
    content_type = r.headers.get("content-type")
    return True if "application/pdf" in content_type else False


def valid_file_path(path):
    r"""validating whether given path is valid or not
    :param path: Path of the pdf file"""
    if os.path.exists(path):
        return True
    return False


def remove_pdf(filename: str):
    r"""REMOVE locally downloaded PDF
    :param filename: filename for the deleting pdf file.
    """
    if os.path.exists(filename):
        os.remove(filename)
        return True
    return False


def process_pdf(filename, metadata, data):
    with open(filename, "rb") as pdf_file:
        pdf_reader = PdfFileReader(pdf_file)
        if metadata:
            data.update({"metadata": {}})
            md = dict(pdf_reader.documentInfo)
            for k, v in md.items():
                data["metadata"][re.sub("[^a-zA-Z0-9]+", "", k)] = v
        data["pages"] = len(pdf_reader.pages)
        data["content"] = [page.extractText() for page in pdf_reader.pages]
    return data


def extract_pdf(url: str = None, path: str = None, metadata: bool = False):
    r"""REMOVE locally downloaded PDF
    :param filename: filename for the deleting pdf file.
    :param metadata: boolean if you want to display available metadata of PDF.
    """

    filename = str(uuid.uuid4().hex) + ".pdf"
    data = {"pages": 0, "content": None}
    if url != None:
        if valid_url(url):
            try:
                download_pdf(url, filename)
                data = process_pdf(filename, metadata, data)
                remove_pdf(filename)
                return data
            except Exception as e:
                remove_pdf(filename)
                raise e
        else:
            print("Invalid URL")

    elif path != None:
        if valid_file_path(path):
            data = process_pdf(path, metadata, data)
            return data
    else:
        print("Invalid URL or File Path")

In [8]:
extract_pdf(url="http://www.africau.edu/images/default/sample.pdf", metadata=False)

DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.